In [ ]:
from google.colab import drive
drive.mount('/content/drive')
path = "drive/MyDrive/'Georgia Tech'/'CS 8803 Human Data Anal'/Project"
%cd $path
%ls

Mounted at /content/drive
/content/drive/MyDrive/Georgia Tech/CS 8803 Human Data Anal/Project
'Copy of Cyberbullying Classification without other cyberbullying label.ipynb'
'Cyberbullying Classification.ipynb'
 cyberbullying_tweets.csv
 ProgressReports.gdoc
 Project.gdoc


In [ ]:
! python --version
! pip install emoji
# ! pip install torchtext==0.12.0
! pip install torchtext
! pip install torch
! pip install transformers

Python 3.7.15
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 240 kB 14.8 MB/s 
  Created wheel for emoji: filename=emoji-2.2.0-py3-none-any.whl size=234926 sha256=7065e9fcc094f778e299a2b311191f6819deea636a8a9daba021c0ec7c3398e2
  Stored in directory: /root/.cache/pip/wheels/f3/e3/f2/1de1c2e3ed742e1df73e0f15d58864e50c7e64f607b548d6cf
Successfully built emoji
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.5 MB 13.6 MB/s 
     |████████████████████████████████| 7.6 MB 47.8 MB/s 
     |████████████████████████████████| 182 kB 69.7 MB/s 


In [ ]:
import torch
import torch.nn as nn
import pandas
import re, string
import emoji
from collections import Counter
import torchtext

from sklearn.metrics import f1_score, accuracy_score, classification_report
from transformers import AutoModel, AutoModelForSequenceClassification, AutoTokenizer, AutoConfig
from torch.utils.data import (DataLoader, RandomSampler, TensorDataset, SequentialSampler, random_split)
import tqdm
import numpy as np

from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'

In [ ]:
data = pandas.read_csv('cyberbullying_tweets.csv')

In [ ]:
data

,tweet_text,cyberbullying_type
0,"In other words #katandandre, your food was cra...",not_cyberbullying
1,Why is #aussietv so white? #MKR #theblock #ImA...,not_cyberbullying
2,@XochitlSuckkks a classy whore? Or more red ve...,not_cyberbullying
3,"@Jason_Gio meh. :P thanks for the heads up, b...",not_cyberbullying
4,@RudhoeEnglish This is an ISIS account pretend...,not_cyberbullying
...,...,...
47687,"Black ppl aren't expected to do anything, depe...",ethnicity
47688,Turner did not withhold his disappointment. Tu...,ethnicity
47689,I swear to God. This dumb nigger bitch. I have...,ethnicity
47690,Yea fuck you RT @therealexel: IF YOURE A NIGGE...,ethnicity


In [ ]:
data.loc[data['cyberbullying_type'] == 'other_cyberbullying']

,tweet_text,cyberbullying_type
23916,"@ikralla fyi, it looks like I was caught by it...",other_cyberbullying
23917,I need to just switch to an organization-based...,other_cyberbullying
23918,RMAed my monoprice. Shoddy power bricks on tho...,other_cyberbullying
23919,@murphy_slaw https://t.co/M8w8xnUnDL,other_cyberbullying
23920,@1Life0Continues i've got the code to interpre...,other_cyberbullying
...,...,...
31734,"@kufr666 @blockbot no, that's @oolon",other_cyberbullying
31735,@AriMelber why are you giving these idiots air...,other_cyberbullying
31736,I am right now watching Enforcers defend Chums...,other_cyberbullying
31737,✨✨✨ misandry is not a word iOS can autocomplet...,other_cyberbullying


In [ ]:
labels = list(data.cyberbullying_type.unique())
labels2id = {label: i for i, label in enumerate(labels)}
print(labels2id)

{'not_cyberbullying': 0, 'gender': 1, 'religion': 2, 'other_cyberbullying': 3, 'age': 4, 'ethnicity': 5}


In [ ]:
data['label'] = data.cyberbullying_type.map(labels2id)

In [ ]:
#Clean emojis from text
def strip_emoji(text):
    return emoji.replace_emoji(text)
    # return re.sub(emoji.get_emoji_regexp(), r"", text) #remove emoji
    
def strip_all_entities(text): 
    text = text.replace('\r', '').replace('\n', ' ').lower() #remove \n and \r and lowercase
    text = re.sub(r"(?:\@|https?\://)\S+", "", text) #remove links and mentions
    text = re.sub(r'[^\x00-\x7f]',r'', text) #remove non utf8/ascii characters such as '\x9a\x91\x97\x9a\x97'
    banned_list= string.punctuation
    table = str.maketrans('', '', banned_list)
    text = text.translate(table)
    text = [word for word in text.split()]
    text = ' '.join(text)
    return text

#remove contractions
def decontract(text):
    text = re.sub(r"can\'t", "can not", text)
    text = re.sub(r"won\'t", "will not", text)
    text = re.sub(r"n\'t", " not", text)
    text = re.sub(r"\'re", " are", text)
    text = re.sub(r"\'s", " is", text)
    text = re.sub(r"\'d", " would", text)
    text = re.sub(r"\'ll", " will", text)
    text = re.sub(r"\'t", " not", text)
    text = re.sub(r"\'ve", " have", text)
    text = re.sub(r"\'m", " am", text)
    return text

def clean_hashtags(tweet):
    # new_tweet = " ".join(word.strip() for word in re.split('#(?!(?:hashtag)\b)[\w-]+(?=(?:\s+#[\w-]+)*\s*$)', tweet)) #remove last hashtags
    new_tweet2 = " ".join(word.strip() for word in re.split('#|_', tweet)) #remove hashtags symbol from words in the middle of the sentence
    return new_tweet2

#Filter special characters such as "&" and "$" present in some words
def filter_chars(a):
    sent = []
    for word in a.split(' '):
        if ('$' in word) | ('&' in word):
            sent.append('')
        else:
            sent.append(word)
    return ' '.join(sent)

#Remove multiple sequential spaces
def remove_mult_spaces(text):
    return re.sub("\s\s+" , " ", text)


#Then we apply all the defined functions in the following order
def deep_clean(text):
    text = strip_emoji(text)
    text = decontract(text)
    text = strip_all_entities(text)
    text = clean_hashtags(text)
    text = filter_chars(text)
    text = remove_mult_spaces(text)
    return text

In [ ]:
PADDING_VALUE = 256
data['cleaned_text'] =  [deep_clean(x).split()[:256] for x in data.tweet_text]

In [ ]:
word_counts = Counter([item for sublist in data['cleaned_text'] for item in sublist])
vocab = torchtext.vocab.vocab(word_counts,
                              min_freq = 5,
                              specials = ['<pad>', '<unk>', ])
vocab.set_default_index(1)

VOCAB_SIZE = len(vocab)

In [ ]:
tokenized_data = [torch.tensor(vocab(x), dtype = torch.int64) for x in data.cleaned_text]

labels = torch.tensor(data.label, dtype = torch.int64)

tokenized_data = nn.utils.rnn.pad_sequence(tokenized_data, batch_first=True)

# bag of words input. frequency of token
bow_tokenized_data = torch.zeros(tokenized_data.shape[0], VOCAB_SIZE)
for i in range(tokenized_data.shape[0]):
  tweet = tokenized_data[i,tokenized_data[i].nonzero().flatten()]
  bow_tokenized_data[i].put_(tweet, torch.ones_like(tweet).float(), accumulate=True)

bow_tokenized_data = nn.utils.rnn.pad_sequence(bow_tokenized_data, batch_first=True)

In [ ]:
TRAIN_SPLIT = int(0.8 * len(labels))
VAL_TEST_SPLIT = len(labels) - TRAIN_SPLIT
VAL_SPLIT = VAL_TEST_SPLIT//2
TEST_SPLIT = VAL_TEST_SPLIT - VAL_SPLIT

bow_cnn_dataset = torch.utils.data.TensorDataset(tokenized_data, labels)
bow_dataset = torch.utils.data.TensorDataset(bow_tokenized_data, labels)

In [ ]:
train, val_test = torch.utils.data.random_split(bow_cnn_dataset, [TRAIN_SPLIT, VAL_TEST_SPLIT])
val, test = torch.utils.data.random_split(val_test, [VAL_SPLIT, TEST_SPLIT])

print(len(train), len(val), len(test))

bow_train, bow_val_test = torch.utils.data.random_split(bow_dataset, [TRAIN_SPLIT, VAL_TEST_SPLIT])
bow_val, bow_test = torch.utils.data.random_split(bow_val_test, [VAL_SPLIT, TEST_SPLIT])

print(len(bow_train), len(bow_val), len(bow_test))

38153 4769 4770
38153 4769 4770


In [ ]:
BATCH_SIZE = 64
train_loader = torch.utils.data.DataLoader(train, BATCH_SIZE)
val_loader = torch.utils.data.DataLoader(val, BATCH_SIZE)
test_loader = torch.utils.data.DataLoader(test, BATCH_SIZE)

bow_train_loader = torch.utils.data.DataLoader(bow_train, BATCH_SIZE)
bow_val_loader = torch.utils.data.DataLoader(bow_val, BATCH_SIZE)
bow_test_loader = torch.utils.data.DataLoader(bow_test, BATCH_SIZE)

# BOW

In [ ]:
class MLP(nn.Module):
  def __init__(self, hidden_dim, dropout, device, outputs):
    super(MLP, self).__init__()

    self.hidden_dim = hidden_dim
    self.dropout = dropout
    self.device = device
    self.outputs = outputs
    
    self.l1 = nn.LazyLinear(hidden_dim).to(self.device)
    self.a1 = nn.ReLU().to(self.device)
    self.d1 = nn.Dropout().to(self.device)
    self.l2 = nn.LazyLinear(self.outputs).to(self.device)

    self.loss = nn.CrossEntropyLoss()
  
  def forward(self, x, y):
    x = self.l1(x).to(self.device)
    x = self.a1(x).to(self.device)
    x = self.d1(x).to(self.device)
    x = self.l2(x).to(self.device)

    loss = self.loss(x, y).to(device)

    return x, loss

  def get_predictions(self, loader):
    with torch.no_grad():
      preds_all = []
      labels_all = []
      accuracies = []

      for tweets, labels in loader:
        tweets = tweets.to(self.device)
        labels = labels.to(self.device)

        self.eval()

        logits, loss = self.forward(tweets, labels)

        _, predictions = torch.max(logits, axis=1)

        preds_all += list(predictions.cpu())
        labels_all += list(labels.cpu())


        accuracy = accuracy_score(list(labels.cpu()), list(predictions.cpu()))
        accuracies.append(accuracy)

    return preds_all, labels_all
  
  def train_model(self, train_loader, val_loader, test_loader, optimizer):
    with tqdm.notebook.tqdm(
      train_loader,
      unit="batch",
      total=len(train_loader)) as batch_iterator:

      total_loss = 0.0
      accuracies = []

      for iteration, data in enumerate(batch_iterator, start=1):
        tweets, labels = data
        tweets = tweets.to(self.device)
        labels = labels.to(self.device)

        optimizer.zero_grad()
        self.zero_grad()
        self.train()

        logits, loss = self.forward(tweets, labels)

        total_loss += loss.item()

        loss.backward()
        optimizer.step()

        _, predictions = torch.max(logits, axis=1)
        accuracy = accuracy_score(list(labels.cpu()), list(predictions.cpu()))
        accuracies.append(accuracy)

        batch_iterator.set_postfix(mean_loss=total_loss / iteration, current_loss=loss.item(), accuracy = accuracy, mean_accuracy = np.mean(accuracies))

        if iteration % 500 == 0:
          pred, label = self.get_predictions(val_loader)
          print("########## VAL ###########")
          print(accuracy_score(pred, label))
          print(f1_score(pred, label, average='micro'))
          print(classification_report(pred, label, zero_division=0, digits = 4),)

          pred, label = self.get_predictions(test_loader)
          print("########## TEST ###########")
          print(accuracy_score(pred, label))
          print(f1_score(pred, label, average='micro'))
          print(classification_report(pred, label, zero_division=0, digits = 4),)
  


In [ ]:
hidden_dim = 128
dropout = 0.
learning_rate = 1e-04
epochs = 15
outputs = len(labels2id.items())

mlp = MLP(hidden_dim, dropout, device, outputs)
optimizer = torch.optim.Adam(mlp.parameters(), lr=learning_rate)

for i in range(epochs):
  mlp.train_model(bow_train_loader, bow_val_loader, bow_test_loader, optimizer)

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/lazy.py:178: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


  0%|          | 0/597 [00:00<?, ?batch/s]

########## VAL ###########
0.7278255399454813
0.7278255399454812
              precision    recall  f1-score   support

           0     0.3974    0.5687    0.4678       582
           1     0.8168    0.6851    0.7451       924
           2     0.9631    0.7568    0.8476      1036
           3     0.2628    0.5791    0.3616       354
           4     0.9795    0.7778    0.8670       981
           5     0.9581    0.8464    0.8988       892

    accuracy                         0.7278      4769
   macro avg     0.7296    0.7023    0.6980      4769
weighted avg     0.8162    0.7278    0.7589      4769

########## TEST ###########
0.7519916142557652
0.7519916142557652
              precision    recall  f1-score   support

           0     0.4486    0.6028    0.5144       579
           1     0.8499    0.7076    0.7723       944
           2     0.9705    0.7685    0.8578      1028
           3     0.2826    0.6364    0.3913       341
           4     0.9694    0.8030    0.8784       985
 

  0%|          | 0/597 [00:00<?, ?batch/s]

########## VAL ###########
0.7982805619626756
0.7982805619626756
              precision    recall  f1-score   support

           0     0.5054    0.5839    0.5418       721
           1     0.7768    0.8776    0.8241       686
           2     0.9558    0.9026    0.9284       862
           3     0.6103    0.5721    0.5906       832
           4     0.9820    0.8968    0.9375       853
           5     0.9708    0.9387    0.9545       815

    accuracy                         0.7983      4769
   macro avg     0.8002    0.7953    0.7961      4769
weighted avg     0.8089    0.7983    0.8021      4769

########## TEST ###########
0.8132075471698114
0.8132075471698114
              precision    recall  f1-score   support

           0     0.5283    0.5888    0.5569       698
           1     0.8028    0.8963    0.8470       704
           2     0.9472    0.9081    0.9272       849
           3     0.6185    0.5814    0.5994       817
           4     0.9767    0.9246    0.9499       862
 

...

########## VAL ###########
0.8280561962675613
0.8280561962675613
              precision    recall  f1-score   support

           0     0.5330    0.6245    0.5751       711
           1     0.8219    0.8934    0.8562       713
           2     0.9472    0.9554    0.9513       807
           3     0.7256    0.5829    0.6465       971
           4     0.9782    0.9682    0.9732       787
           5     0.9759    0.9859    0.9809       780

    accuracy                         0.8281      4769
   macro avg     0.8303    0.8351    0.8305      4769
weighted avg     0.8314    0.8281    0.8274      4769

########## TEST ###########
0.8412997903563941
0.8412997903563941
              precision    recall  f1-score   support

           0     0.5347    0.6380    0.5818       652
           1     0.8499    0.9027    0.8755       740
           2     0.9459    0.9589    0.9524       803
           3     0.7409    0.6008    0.6636       947
           4     0.9730    0.9683    0.9707       820
 

  0%|          | 0/597 [00:00<?, ?batch/s]

########## VAL ###########
0.8276368211365066
0.8276368211365066
              precision    recall  f1-score   support

           0     0.5414    0.6187    0.5775       729
           1     0.8258    0.8914    0.8573       718
           2     0.9484    0.9566    0.9525       807
           3     0.7090    0.5833    0.6400       948
           4     0.9795    0.9670    0.9732       789
           5     0.9746    0.9871    0.9808       778

    accuracy                         0.8276      4769
   macro avg     0.8298    0.8340    0.8302      4769
weighted avg     0.8296    0.8276    0.8268      4769

########## TEST ###########
0.8412997903563941
0.8412997903563941
              precision    recall  f1-score   support

           0     0.5437    0.6295    0.5834       672
           1     0.8550    0.8984    0.8761       748
           2     0.9459    0.9589    0.9524       803
           3     0.7240    0.6057    0.6595       918
           4     0.9730    0.9683    0.9707       820
 

  0%|          | 0/597 [00:00<?, ?batch/s]

########## VAL ###########
0.8280561962675613
0.8280561962675613
              precision    recall  f1-score   support

           0     0.5414    0.6204    0.5782       727
           1     0.8310    0.8932    0.8610       721
           2     0.9459    0.9553    0.9506       806
           3     0.7077    0.5823    0.6389       948
           4     0.9807    0.9671    0.9739       790
           5     0.9746    0.9884    0.9815       777

    accuracy                         0.8281      4769
   macro avg     0.8302    0.8344    0.8307      4769
weighted avg     0.8300    0.8281    0.8272      4769

########## TEST ###########
0.8419287211740042
0.8419287211740042
              precision    recall  f1-score   support

           0     0.5386    0.6329    0.5819       662
           1     0.8601    0.9025    0.8808       749
           2     0.9459    0.9613    0.9536       801
           3     0.7305    0.6013    0.6596       933
           4     0.9718    0.9694    0.9706       818
 

# CNN

In [ ]:
from torch.nn.modules.conv import Conv1d
class Conv1dBlock(nn.Module):
  def __init__(self, in_channels, out_channels, kernel_size=2, padding=0, max_pool=2, dropout=0.1):
    super(Conv1dBlock, self).__init__()

    self.max_pool = max_pool
    self.l1 = nn.Conv1d(in_channels, out_channels, kernel_size=kernel_size, stride=1, padding = padding)
    self.l2 = nn.ReLU()
    self.l3 = nn.MaxPool1d(kernel_size=self.max_pool, stride=1, padding=0)
    self.l4 = nn.Dropout(dropout)
  
  def __call__(self, x):
    x = self.l1(x)
    x = self.l2(x)
    if self.max_pool > 0:
      x = self.l3(x)
    x = self.l4(x)
    return x

class ConvNet(nn.Module):
  def __init__(self, hidden_dim, dropout, device, outputs, embedding_dim, kernel_size=2, max_pool=2):
    super(ConvNet, self).__init__()

    self.conv_dim = conv_dim
    self.hidden_dim = hidden_dim
    self.dropout = dropout
    self.device = device
    self.outputs = outputs
    self.embedding_dim = embedding_dim
    self.max_pool = max_pool
    self.kernel_size = kernel_size

    self.e = nn.Embedding(num_embeddings = VOCAB_SIZE, embedding_dim=self.embedding_dim).to(self.device)

    self.c1 = Conv1dBlock(in_channels=self.embedding_dim, out_channels=self.conv_dim, kernel_size=self.kernel_size, padding=0, max_pool=self.max_pool, dropout=0.1).to(self.device)
    self.c2 = Conv1dBlock(in_channels=self.conv_dim, out_channels=self.conv_dim, kernel_size=self.kernel_size, padding=0, max_pool=self.max_pool, dropout=0.1).to(self.device)
    self.c3 = Conv1dBlock(in_channels=self.conv_dim, out_channels=self.conv_dim, kernel_size=self.kernel_size, padding=0, max_pool=self.max_pool, dropout=0.1).to(self.device)

    self.l1 = nn.LazyLinear(hidden_dim).to(self.device)
    self.a1 = nn.ReLU().to(self.device)
    self.d1 = nn.Dropout().to(self.device)
    self.l2 = nn.LazyLinear(self.outputs).to(self.device)

    self.loss = nn.CrossEntropyLoss()
  
  def forward(self, x, y):
    embeddings = self.e(x)
    C, L, I = embeddings.shape
    x = embeddings.reshape(C, I, L)
    x = self.c1(x).to(self.device)
    x = self.c2(x).to(self.device)
    x = self.c3(x).to(self.device)
    x = x.flatten(1)
    x = self.l1(x).to(self.device)
    x = self.a1(x).to(self.device)
    x = self.d1(x).to(self.device)
    x = self.l2(x).to(self.device)
    # x = x[:,0,:]
    loss = self.loss(x, y).to(device)

    return x, loss

  def get_predictions(self, loader):
    with torch.no_grad():
      preds_all = []
      labels_all = []
      accuracies = []

      for tweets, labels in loader:
        tweets = tweets.to(self.device)
        labels = labels.to(self.device)

        self.eval()

        logits, loss = self.forward(tweets, labels)

        _, predictions = torch.max(logits, axis=1)

        preds_all += list(predictions.cpu())
        labels_all += list(labels.cpu())


        accuracy = accuracy_score(list(labels.cpu()), list(predictions.cpu()))
        accuracies.append(accuracy)

    return preds_all, labels_all
  
  def train_model(self, train_loader, val_loader, test_loader, optimizer):
    with tqdm.notebook.tqdm(
      train_loader,
      unit="batch",
      total=len(train_loader)) as batch_iterator:

      total_loss = 0.0
      accuracies = []

      for iteration, data in enumerate(batch_iterator, start=1):
        tweets, labels = data
        tweets = tweets.to(self.device)
        labels = labels.to(self.device)

        optimizer.zero_grad()
        self.zero_grad()
        self.train()


        logits, loss = self.forward(tweets, labels)

        total_loss += loss.item()

        loss.backward()
        optimizer.step()

        _, predictions = torch.max(logits, axis=1)
        accuracy = accuracy_score(list(labels.cpu()), list(predictions.cpu()))
        accuracies.append(accuracy)

        batch_iterator.set_postfix(mean_loss=total_loss / iteration, current_loss=loss.item(), accuracy = accuracy, mean_accuracy = np.mean(accuracies))

        if iteration % 500 == 0:
          pred, label = self.get_predictions(val_loader)
          print("########## VAL ###########")
          print(accuracy_score(pred, label))
          print(f1_score(pred, label, average='micro'))
          print(classification_report(pred, label, zero_division=0, digits = 4),)

          pred, label = self.get_predictions(test_loader)
          print("########## TEST ###########")
          print(accuracy_score(pred, label))
          print(f1_score(pred, label, average='micro'))
          print(classification_report(pred, label, zero_division=0, digits = 4),)
  


In [ ]:

conv_dim = 32
hidden_dim = 128
dropout = 0.1
learning_rate = 5e-04
epochs = 15
outputs = len(labels2id.items())
embedding_dim = 128

convnet = ConvNet(hidden_dim, dropout, device, outputs, embedding_dim, kernel_size=8, max_pool=2)
optimizer = torch.optim.Adam(convnet.parameters(), lr=learning_rate)

for i in range(epochs):
  convnet.train_model(train_loader, val_loader, test_loader, optimizer)

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/lazy.py:178: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


  0%|          | 0/597 [00:00<?, ?batch/s]

########## VAL ###########
0.6462570769553365
0.6462570769553365
              precision    recall  f1-score   support

           0     0.1722    0.4576    0.2502       295
           1     0.5675    0.7346    0.6403       584
           2     0.8087    0.6176    0.7004      1020
           3     0.6544    0.3875    0.4868      1285
           4     0.8240    0.8993    0.8600       765
           5     0.8220    0.8561    0.8387       820

    accuracy                         0.6463      4769
   macro avg     0.6415    0.6588    0.6294      4769
weighted avg     0.7030    0.6463    0.6570      4769

########## TEST ###########
0.6438155136268344
0.6438155136268344
              precision    recall  f1-score   support

           0     0.1351    0.3919    0.2009       273
           1     0.5635    0.7238    0.6336       601
           2     0.8395    0.6471    0.7308      1091
           3     0.6627    0.3870    0.4886      1305
           4     0.8389    0.8827    0.8602       767
 

  0%|          | 0/597 [00:00<?, ?batch/s]

########## VAL ###########
0.7250996015936255
0.7250996015936254
              precision    recall  f1-score   support

           0     0.2742    0.4555    0.3424       472
           1     0.7090    0.7813    0.7434       686
           2     0.7972    0.9159    0.8524       678
           3     0.7096    0.4323    0.5373      1249
           4     0.9269    0.9337    0.9303       829
           5     0.9040    0.9029    0.9035       855

    accuracy                         0.7251      4769
   macro avg     0.7202    0.7370    0.7182      4769
weighted avg     0.7515    0.7251    0.7264      4769

########## TEST ###########
0.7132075471698113
0.7132075471698113
              precision    recall  f1-score   support

           0     0.2058    0.4201    0.2763       388
           1     0.7047    0.7673    0.7346       709
           2     0.8193    0.9054    0.8602       761
           3     0.7087    0.4144    0.5230      1303
           4     0.9219    0.9129    0.9174       815
 

...

########## VAL ###########
0.8135877542461732
0.8135877542461732
              precision    recall  f1-score   support

           0     0.4235    0.6182    0.5026       537
           1     0.7923    0.8927    0.8395       671
           2     0.9166    0.9584    0.9370       745
           3     0.8121    0.5323    0.6431      1161
           4     0.9677    0.9608    0.9642       841
           5     0.9473    0.9939    0.9700       814

    accuracy                         0.8136      4769
   macro avg     0.8099    0.8260    0.8094      4769
weighted avg     0.8324    0.8136    0.8133      4769

########## TEST ###########
0.7966457023060797
0.7966457023060797
              precision    recall  f1-score   support

           0     0.3447    0.5594    0.4266       488
           1     0.8018    0.8906    0.8439       695
           2     0.9132    0.9423    0.9275       815
           3     0.8018    0.5083    0.6222      1202
           4     0.9628    0.9557    0.9593       813
 

  0%|          | 0/597 [00:00<?, ?batch/s]

########## VAL ###########
0.8154749423359194
0.8154749423359194
              precision    recall  f1-score   support

           0     0.4005    0.6528    0.4964       481
           1     0.8214    0.8460    0.8336       734
           2     0.9191    0.9598    0.9390       746
           3     0.8016    0.5365    0.6428      1137
           4     0.9725    0.9621    0.9672       844
           5     0.9555    0.9867    0.9709       827

    accuracy                         0.8155      4769
   macro avg     0.8118    0.8240    0.8083      4769
weighted avg     0.8395    0.8155    0.8180      4769

########## TEST ###########
0.8006289308176101
0.8006289308176101
              precision    recall  f1-score   support

           0     0.3270    0.5860    0.4198       442
           1     0.8381    0.8414    0.8397       769
           2     0.9156    0.9471    0.9311       813
           3     0.7874    0.5204    0.6266      1153
           4     0.9703    0.9526    0.9613       822
 

  0%|          | 0/597 [00:00<?, ?batch/s]

########## VAL ###########
0.8175718179911932
0.8175718179911932
              precision    recall  f1-score   support

           0     0.4707    0.6059    0.5298       609
           1     0.7963    0.8879    0.8396       678
           2     0.9101    0.9659    0.9372       734
           3     0.7884    0.5500    0.6479      1091
           4     0.9629    0.9663    0.9646       832
           5     0.9543    0.9879    0.9708       825

    accuracy                         0.8176      4769
   macro avg     0.8138    0.8273    0.8150      4769
weighted avg     0.8268    0.8176    0.8157      4769

########## TEST ###########
0.8020964360587002
0.8020964360587003
              precision    recall  f1-score   support

           0     0.3927    0.5544    0.4597       561
           1     0.8018    0.8906    0.8439       695
           2     0.9108    0.9422    0.9262       813
           3     0.7717    0.5255    0.6252      1119
           4     0.9641    0.9617    0.9629       809
 

# Transformer Models

In [ ]:
data

,tweet_text,cyberbullying_type,label,cleaned_text
0,"In other words #katandandre, your food was cra...",not_cyberbullying,0,"[in, other, words, katandandre, your, food, wa..."
1,Why is #aussietv so white? #MKR #theblock #ImA...,not_cyberbullying,0,"[why, is, aussietv, so, white, mkr, theblock, ..."
2,@XochitlSuckkks a classy whore? Or more red ve...,not_cyberbullying,0,"[a, classy, whore, or, more, red, velvet, cupc..."
3,"@Jason_Gio meh. :P thanks for the heads up, b...",not_cyberbullying,0,"[meh, p, thanks, for, the, heads, up, but, not..."
4,@RudhoeEnglish This is an ISIS account pretend...,not_cyberbullying,0,"[this, is, an, isis, account, pretending, to, ..."
...,...,...,...,...
47687,"Black ppl aren't expected to do anything, depe...",ethnicity,5,"[black, ppl, are, not, expected, to, do, anyth..."
47688,Turner did not withhold his disappointment. Tu...,ethnicity,5,"[turner, did, not, withhold, his, disappointme..."
47689,I swear to God. This dumb nigger bitch. I have...,ethnicity,5,"[i, swear, to, god, this, dumb, nigger, bitch,..."
47690,Yea fuck you RT @therealexel: IF YOURE A NIGGE...,ethnicity,5,"[yea, fuck, you, rt, if, youre, a, nigger, fuc..."


In [ ]:
BATCH_SIZE = 50
del convnet
del mlp

In [ ]:
model_type = 'bert-base-uncased'
tokenizer = AutoTokenizer.from_pretrained(model_type, use_fast=False)

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

In [ ]:
tokenized_text = list(data['cleaned_text'])
tokenized_text = [' '.join(x) for x in tokenized_text]
tokenized_text = tokenizer(tokenized_text, padding = 'max_length', max_length = PADDING_VALUE, truncation=True, add_special_tokens=True)
x_ids = torch.tensor([f for f in tokenized_text.input_ids], dtype=torch.long)
x_mask = torch.tensor([f for f in tokenized_text.attention_mask], dtype=torch.long)
label_tensor = torch.tensor(data.label)

In [ ]:
tensor_dataset = TensorDataset(x_ids, x_mask, label_tensor)

In [ ]:
train, val_test = random_split(tensor_dataset, [TRAIN_SPLIT, VAL_TEST_SPLIT], generator=torch.Generator().manual_seed(13))
val, test = random_split(val_test, [VAL_SPLIT, TEST_SPLIT], generator=torch.Generator().manual_seed(13))

print(len(train), len(val), len(test))

38153 4769 4770


In [ ]:
train_loader = DataLoader(train, batch_size=BATCH_SIZE)
val_loader = DataLoader(val, batch_size=BATCH_SIZE)
test_loader = DataLoader(test, batch_size=BATCH_SIZE)

In [ ]:
class transformer_classifier(nn.Module):
  def __init__(self, model_type, hidden_dim, dropout, device ):
      super(transformer_classifier, self).__init__()

      self.device = device
      self.hidden_dim = hidden_dim
      self.LABELS_NUM = len(labels2id.items())

      self.lm = AutoModel.from_pretrained(model_type).to(self.device)

      config = config = AutoConfig.from_pretrained(model_type, num_labels = self.LABELS_NUM)
      self.lm_sequence_classification = AutoModelForSequenceClassification.from_pretrained(model_type, config=config).to(self.device)

      self.l1 = nn.LazyLinear(hidden_dim).to(self.device)
      self.a1 = nn.GELU().to(self.device)
      self.d1 = nn.Dropout(dropout).to(self.device)
      self.l2 = nn.LazyLinear(self.LABELS_NUM).to(self.device)

      self.loss = nn.CrossEntropyLoss()


  def forward(self, ids, masks, labels, mode):
    if mode:
      output = self.lm_sequence_classification(ids, masks, labels=labels)
      return output.logits.to(self.device), output.loss.to(self.device)
    else:
      x = self.lm(ids, masks).last_hidden_state[:, 0].to(self.device)
      x = self.l1(x).to(self.device)
      x = self.a1(x).to(self.device)
      x = self.d1(x).to(self.device)
      x = self.l2(x).to(self.device)

      loss = self.loss(x, labels).to(self.device)

      return x, loss

  def get_predictions(self, loader, mode):
    with torch.no_grad():
      with tqdm.notebook.tqdm(
      loader,
      unit="batch",
      total=len(loader)) as batch_iterator:

        preds_all = []
        labels_all = []
        accuracies = []

        for iteration, data in enumerate(batch_iterator, start=1):
          ids, masks, labels = data
          ids = ids.to(self.device)
          masks = masks.to(self.device)
          labels = labels.to(self.device)

          self.eval()

          logits, loss = self.forward(ids, masks, labels, mode)

          _, predictions = torch.max(logits, axis=1)

          preds_all += list(predictions.cpu())
          labels_all += list(labels.cpu())


          accuracy = accuracy_score(list(labels.cpu()), list(predictions.cpu()))
          accuracies.append(accuracy)

          batch_iterator.set_postfix(accuracy = accuracy, mean_accuracy = np.mean(accuracies))
    return preds_all, labels_all

  def train_model(self, train_loader, val_loader, test_loader, mode, optimizer):
    with tqdm.notebook.tqdm(
      train_loader,
      unit="batch",
      total=len(train_loader)) as batch_iterator:

      total_loss = 0.0
      accuracies = []

      for iteration, data in enumerate(batch_iterator, start=1):
        ids, masks, labels = data
        ids = ids.to(self.device)
        masks = masks.to(self.device)
        labels = labels.to(self.device)

        optimizer.zero_grad()
        self.zero_grad()
        self.train()

        logits, loss = self.forward(ids, masks, labels, mode)

        total_loss += loss.item()

        loss.backward()
        optimizer.step()

        _, predictions = torch.max(logits, axis=1)
        accuracy = accuracy_score(list(labels.cpu()), list(predictions.cpu()))
        accuracies.append(accuracy)

        batch_iterator.set_postfix(mean_loss=total_loss / iteration, current_loss=loss.item(), accuracy = accuracy, mean_accuracy = np.mean(accuracies))

        if iteration % 400 == 0:
          pred, label = self.get_predictions(val_loader, mode)
          print("########## VAL ###########")
          print(accuracy_score(pred, label))
          print(f1_score(pred, label, average='micro'))
          print(classification_report(pred, label, zero_division=0, digits = 4),)

          pred, label = self.get_predictions(test_loader, mode)
          print("########## TEST ###########")
          print(accuracy_score(pred, label))
          print(f1_score(pred, label, average='micro'))
          print(classification_report(pred, label, zero_division=0, digits = 4),)

          



In [ ]:
hidden_dim = 128
dropout = 0.2
learning_rate = 1e-04
epochs = 5

classifier = transformer_classifier(model_type, hidden_dim, dropout, device)
optimizer = torch.optim.Adam(classifier.parameters(), lr=learning_rate)

for i in range(epochs):
  classifier.train_model(train_loader, val_loader, test_loader, False, optimizer)

classifier = transformer_classifier(model_type, hidden_dim, dropout, device)
optimizer = torch.optim.Adam(classifier.parameters(), lr=learning_rate)

for i in range(epochs):
  classifier.train_model(train_loader, val_loader, test_loader, True, optimizer)

Downloading:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.weight',

  0%|          | 0/764 [00:00<?, ?batch/s]

  0%|          | 0/96 [00:00<?, ?batch/s]

########## VAL ###########
0.8490249528202978
0.8490249528202978
              precision    recall  f1-score   support

           0     0.5997    0.6478    0.6228       724
           1     0.8310    0.9115    0.8694       723
           2     0.9699    0.9361    0.9527       861
           3     0.7174    0.6315    0.6717       844
           4     0.9740    0.9842    0.9791       760
           5     0.9789    0.9732    0.9760       857

    accuracy                         0.8490      4769
   macro avg     0.8451    0.8474    0.8453      4769
weighted avg     0.8502    0.8490    0.8487      4769



  0%|          | 0/96 [00:00<?, ?batch/s]

########## TEST ###########
0.8392033542976939
0.8392033542976939
              precision    recall  f1-score   support

           0     0.5961    0.6380    0.6164       768
           1     0.8260    0.8908    0.8571       714
           2     0.9684    0.9457    0.9570       811
           3     0.7029    0.6095    0.6529       881
           4     0.9628    0.9926    0.9775       809
           5     0.9772    0.9784    0.9778       787

    accuracy                         0.8392      4770
   macro avg     0.8389    0.8425    0.8398      4770
weighted avg     0.8386    0.8392    0.8379      4770



KeyboardInterrupt: ignored

In [ ]:
model_type

'bert-base-uncased'

In [ ]:
model_type = 'distilbert-base-uncased'
tokenizer = AutoTokenizer.from_pretrained(model_type, use_fast=False)
tokenized_text = list(data['cleaned_text'])
tokenized_text = [' '.join(x) for x in tokenized_text]
tokenized_text = tokenizer(tokenized_text, padding = 'max_length', max_length = PADDING_VALUE, truncation=True, add_special_tokens=True)
x_ids = torch.tensor([f for f in tokenized_text.input_ids], dtype=torch.long)
x_mask = torch.tensor([f for f in tokenized_text.attention_mask], dtype=torch.long)
label_tensor = torch.tensor(data.label)
tensor_dataset = TensorDataset(x_ids, x_mask, label_tensor)
train, val_test = random_split(tensor_dataset, [TRAIN_SPLIT, VAL_TEST_SPLIT], generator=torch.Generator().manual_seed(13))
val, test = random_split(val_test, [VAL_SPLIT, TEST_SPLIT], generator=torch.Generator().manual_seed(13))
train_loader = DataLoader(train, batch_size=BATCH_SIZE)
val_loader = DataLoader(val, batch_size=BATCH_SIZE)
test_loader = DataLoader(test, batch_size=BATCH_SIZE)

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/483 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

In [ ]:
classifier = transformer_classifier(model_type, hidden_dim, dropout, device)
optimizer = torch.optim.Adam(classifier.parameters(), lr=learning_rate)

for i in range(epochs):
  classifier.train_model(train_loader, val_loader, test_loader, False, optimizer)

classifier = transformer_classifier(model_type, hidden_dim, dropout, device)
optimizer = torch.optim.Adam(classifier.parameters(), lr=learning_rate)

for i in range(epochs):
  classifier.train_model(train_loader, val_loader, test_loader, True, optimizer)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_transform.bias', 'vocab_projector.weight', 'vocab_layer_norm.bias', 'vocab_layer_norm.weight', 'vocab_projector.bias', 'vocab_transform.weight']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.bias', 'vocab_projector.weight', 'vocab_layer_norm.bias', 'vocab_layer_norm.weight', 'vocab_projector.bias', 'vocab_tr

  0%|          | 0/764 [00:00<?, ?batch/s]

  0%|          | 0/96 [00:00<?, ?batch/s]

########## VAL ###########
0.8458796393373873
0.8458796393373873
              precision    recall  f1-score   support

           0     0.4335    0.7618    0.5526       445
           1     0.8663    0.8752    0.8707       785
           2     0.9699    0.9641    0.9670       836
           3     0.8439    0.5700    0.6804      1100
           4     0.9688    0.9854    0.9770       755
           5     0.9754    0.9800    0.9776       848

    accuracy                         0.8459      4769
   macro avg     0.8430    0.8561    0.8376      4769
weighted avg     0.8745    0.8459    0.8499      4769



  0%|          | 0/96 [00:00<?, ?batch/s]

...

  0%|          | 0/764 [00:00<?, ?batch/s]

  0%|          | 0/96 [00:00<?, ?batch/s]

########## VAL ###########
0.8332983854057454
0.8332983854057454
              precision    recall  f1-score   support

           0     0.5013    0.6049    0.5483       648
           1     0.8865    0.8690    0.8777       809
           2     0.9651    0.9548    0.9599       840
           3     0.6608    0.5644    0.6088       870
           4     0.9753    0.9868    0.9810       759
           5     0.9824    0.9929    0.9876       843

    accuracy                         0.8333      4769
   macro avg     0.8286    0.8288    0.8272      4769
weighted avg     0.8379    0.8333    0.8342      4769



  0%|          | 0/96 [00:00<?, ?batch/s]

########## TEST ###########
0.8314465408805032
0.8314465408805032
              precision    recall  f1-score   support

           0     0.5012    0.6319    0.5590       652
           1     0.8831    0.8651    0.8740       786
           2     0.9634    0.9443    0.9537       808
           3     0.6976    0.5694    0.6271       936
           4     0.9748    0.9903    0.9825       821
           5     0.9708    0.9974    0.9839       767

    accuracy                         0.8314      4770
   macro avg     0.8318    0.8331    0.8300      4770
weighted avg     0.8380    0.8314    0.8324      4770

